In [1]:
# Lab 10 MNIST and softmax
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

import numpy as np
import os
import copy
import datetime

# torch.set_printoptions(profile="full")
# np.set_printoptions(threshold=np.inf)


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(111)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 5
batch_size = 100

In [4]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)
test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [6]:
class MyNet(torch.nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 25, kernel_size=3),
            torch.nn.BatchNorm2d(25),
            torch.nn.ReLU(inplace=True)
        )
 
        self.layer2 = torch.nn.Sequential(
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
 
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(25, 50, kernel_size=3),
            torch.nn.BatchNorm2d(50),
            torch.nn.ReLU(inplace=True)
        )
 
        self.layer4 = torch.nn.Sequential(
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
 
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(50 * 5 * 5, 1024),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(1024, 128),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(128, 10)
        )
 
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

model = MyNet().to(device)
print(model)

MyNet(
  (layer1): Sequential(
    (0): Conv2d(1, 25, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer2): Sequential(
    (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(25, 50, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer4): Sequential(
    (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=1250, out_features=1024, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=1024, out_features=128, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [7]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
# total_batch = len(data_loader)
# model.train()
# for epoch in range(training_epochs):
#     avg_cost = 0

#     for X, Y in data_loader:
#         X = torch.autograd.Variable(X).to(device)
#         Y = Y.to(device)

#         optimizer.zero_grad()
#         hypothesis = model(X)
#         cost = criterion(hypothesis, Y)
#         cost.backward()
#         optimizer.step()

#         avg_cost += cost / total_batch

#     print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

# print('Learning finished')

In [17]:
filename = "merge_attack0normal5_05-05--16-57-13.pth"
model.load_state_dict(torch.load("./merge_save/"+filename)["state_dict"])

<All keys matched successfully>

In [9]:
parm = {}
for name,parameters in model.named_parameters():
    parm[name]=parameters
    print(name)
    # print(parm[name])

layer1.0.weight
layer1.0.bias
layer1.1.weight
layer1.1.bias
layer3.0.weight
layer3.0.bias
layer3.1.weight
layer3.1.bias
fc.0.weight
fc.0.bias
fc.2.weight
fc.2.bias
fc.4.weight
fc.4.bias


In [11]:
# parm["fc.4.weight"]

In [14]:
# tensors = []

# model.eval()
# with torch.no_grad():

#     t=10
#     for i in range(0,t):
#         r = random.randint(0, (batch_size) - 1)

#         for X_test, Y_test in test_loader:
#             X_test = X_test.to(device)
#             Y_test = Y_test.to(device)
        

#         X_single_data = X_test[r]
#         Y_single_data = Y_test[r]

#         # print(X_single_data)
        
#         layerP = model.layer1(torch.unsqueeze(X_single_data,dim=0))
#         # print("1",layerP)
#         layerP = model.layer2(layerP)
#         # print("2",layerP)
#         layerP = model.layer3(layerP)
#         # print("3",layerP)
#         layerP = model.layer4(layerP)
#         layerP = layerP.detach().squeeze().reshape(-1,1250)

#         tensor1 = torch.mm(layerP, parm['fc.0.weight'].data.permute(1,0)) + parm['fc.0.bias']
#         tensor1 = torch.mm(tensor1, parm['fc.2.weight'].data.permute(1,0)) + parm['fc.2.bias']
        
#         tensors.append(tensor1.cpu().detach().numpy()[0])

In [13]:
parm['fc.4.weight'].shape

torch.Size([10, 128])

In [18]:
dt = datetime.datetime.now()
# np.savetxt("./misc/vr_metric"+dt.strftime("%d--%H-%M-%S")+".dat", vp+vp.T, fmt="%1.5f")
np.savetxt("./metric/param="+filename+dt.strftime("%m-%d--%H-%M-%S")+".dat", parm['fc.4.weight'].detach().cpu(), fmt="%1.5f")

In [14]:
nb = np.linspace(0,0,128)
for i in tensors:
    nb += i
nb /= t

ns = np.linspace(0,0,128)
vp = np.linspace(0,0,128*128).reshape(128,128)
vp.shape

(128, 128)

In [15]:
for i in range(0,len(ns)):
    for nps in tensors:
        ns[i] += nps[i] ** 2
    ns[i] = ns[i] ** 0.5

In [16]:

for i in range(0,len(ns)):
    for j in range(i+1,len(ns)):
        for nps in tensors:
            vp[i,j] += (nps[i] - nb[i]) * (nps[j] - nb[j])
        vp[i,j] /= (ns[i] * ns[j])

print(vp + vp.T)
    

[[ 0.         -0.02144168  0.04302791 ... -0.03783377  0.03493989
  -0.00821831]
 [-0.02144168  0.         -0.02036495 ...  0.01806049  0.00460479
  -0.00194868]
 [ 0.04302791 -0.02036495  0.         ... -0.0364618   0.03285496
  -0.02189673]
 ...
 [-0.03783377  0.01806049 -0.0364618  ...  0.         -0.03127106
   0.01378983]
 [ 0.03493989  0.00460479  0.03285496 ... -0.03127106  0.
  -0.00969969]
 [-0.00821831 -0.00194868 -0.02189673 ...  0.01378983 -0.00969969
   0.        ]]


In [ ]:
# f = open("./misc/vr_metric",'w')
# f.write(str(vp+vp.T))
# f.flush()
# f.close()

In [16]:
dt = datetime.datetime.now()
# np.savetxt("./misc/vr_metric"+dt.strftime("%d--%H-%M-%S")+".dat", vp+vp.T, fmt="%1.5f")
np.savetxt("./myfed_normal_save/vr_metric="+filename+dt.strftime("%m-%d--%H-%M-%S")+".dat", vp+vp.T, fmt="%1.5f")

NameError: name 'filename' is not defined